<h3> 1. Retrieve the Data Source <h3>

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2024-01-10 00:52:27--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2,6K) [application/zip]
Saving to: 'source.zip'

     0K ..                                                    100%  506M=0s

2024-01-10 00:52:29 (506 MB/s) - 'source.zip' saved [2707/2707]



In [3]:
#unzip the zip file
import zipfile
with zipfile.ZipFile('source.zip', 'r') as zip_ref:
    zip_ref.extractall()

<h3> 2. Import necessary libraries <h3>

In [1]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime 

In [2]:
#declare variables that state file name:
log_file = "log_file.txt" 
target_file = "transformed_data.csv" 

<h3>3. Write functions for Extract stage <h3>

In [3]:
#CSV
def extract_from_csv (input_file):
    df = pd.read_csv(input_file)
    return df

In [4]:
#JSON
def extract_from_json (input_file):
    df = pd.read_json(input_file, lines = True) #Notice how json file is oschestrated as below, in turn, we need to add arg lines = True

<img src="jsonsample.png" alt="Alternative text" />

In [5]:
#XML
def extract_from_xml (input_file):
    dataframe = pd.DataFrame (columns=["name", "height", "weight"]) #blueprint the columns for DF
    tree= ET.parse(input_file) #turn input_file as xml to an instance of ET
    data = tree.getroot() #get into data ticket
    for person in data:
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height": height, "weight":weight}])], ignore_index=True) #need to be the same name from dataframe which was declared below
    return dataframe

<img src="xmlsample.png" alt="Alternative text" />

In [6]:
#extract all:
def extract():
    extracted_data = pd.DataFrame(columns =["name", "height", "weight"])
    for csvfile in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index = True)
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index = True)
    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index = True)
    #after extract data from all kinds of files 
    return extracted_data

In [11]:
#Test how data is extracted
test_extracted = extract()
test_extracted['height'] #type: pandas.core.series.Series

0     65.78
1     71.52
2     69.40
3     68.22
4     67.79
5     65.78
6     71.52
7     69.40
8     68.22
9     67.79
10    65.78
11    71.52
12    69.40
13    68.22
14    67.79
15    67.90
16    66.78
17    66.49
18    67.62
19    67.90
20    66.78
21    66.49
22    67.62
23    67.90
24    66.78
25    66.49
26    67.62
Name: height, dtype: float64

<h3>4. Transform function <h3>

In [12]:
def transform (data):
    data['height'] = round(data.height *0.0254, 2) #data.height is also pandas.core.series.Series
    data['weight'] = round(data.weight * 0.45359237, 2)
    return data

<h3> 5. Load function <h3>

In [13]:
def load_data (target_file, transformed_data):
    transformed_data.to_csv(target_file)

<h3>6. Log function <h3>

In [17]:
def log_progress (message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file, 'a') as file:
        file.write ('At' + timestamp + ',' + message + '\n')

<h3> Test <>

In [18]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15  simon    1.72   50.97
16  jacob    1.70   54.73
17  cindy    1.69   57.81
18   ivan    1.72   51.77
19  simon    1.72   50.97
20  jacob    1.70   54.73
21  cindy    1.69   57.81
22   ivan    1.72   51.77
23  simon    1.72   50.97
24  jacob    1.70   54.73
25  cindy    1.69   57.81
26   ivan    1.72   51.77
